# 1. Libraries Import

In [ ]:
# ========================================================
# = Libraries import
# ========================================================

from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import boto3
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import plotly.express as px
from timezonefinder import TimezoneFinder
import pytz
import datetime
import math

# 2. AWS credentials

In [ ]:
# ========================================================
# = AWS Credentials
# ========================================================

PROD_AWS_PROFILE = "gsesami-prod"
AWS_REGION = "us-west-2"

prod_session = boto3.session.Session(profile_name=PROD_AWS_PROFILE)

prod_client = prod_session.client(
    "timestream-query", region_name=AWS_REGION)

# 3. Querying TimeStream

## 3.1. Monitor ID, Site ID, and time period

In [ ]:
# Monitor ID
MID = '23779471'

In [ ]:
sites_list = pd.read_csv('./input_data/Site_List.csv')
monitors_list = pd.read_csv('./input_data/monitors/Monitors_List.csv')
# monitors_faulty = pd.read_csv('./faulty_sites/window_size_7days/monitors_faulty_sites.csv')

In [ ]:
# Get Side ID:
site_id = monitors_list.loc[monitors_list['source'] == str("MNTR|" + MID), 'siteId'].iloc[0]

# When working with MONITORS, we determine a time period:
time_start = '2022-09-01'
time_end = '2022-10-01'

# When working with SITES, we determine a date period:
date_start = '2022-01-01'
date_end = '2023-02-01'

# Checking timezone
timezone_value = 'Australia/Sydney'
timezone_value = sites_list[sites_list['source'] == site_id].iloc[0]['timezone']

## THIS IS PART OF YANS CODE: TO DOUBLE CHECK:
# time_endtz = datetime.datetime.fromisoformat(time_end_short)
time_starttz = pytz.timezone('UTC').localize(datetime.datetime.strptime(time_start, '%Y-%m-%d'))
time_endtz = pytz.timezone('UTC').localize(datetime.datetime.strptime(time_end, '%Y-%m-%d'))

In [ ]:
print("Site ID is : ", site_id)

# 3.2 Helper functions to read metrics

In [ ]:
def read_metric(time_start, time_end, measure_name, MID):
    timeid = []
    data_values = []
    ##----------------- read the Performance  --------------##
    query = """SELECT time, measure_value::bigint
                    FROM "GSESTimeseries"."GSESTimeseriesTable"
                    WHERE measure_name = '""" + measure_name + """'
                    AND MID = '""" + MID + """'
                    AND time BETWEEN '""" + time_start + """'
                    AND '""" + time_end + """' """

    client = prod_client
    paginator = client.get_paginator("query")
    page_iterator = paginator.paginate(QueryString=query,)
    i = 1
    for page in page_iterator:
        # print(page)
        try:
            timeid_page = [f[0]['ScalarValue'] for f in pd.DataFrame(page["Rows"])['Data']]
            data_values_page = [f[1]['ScalarValue'] for f in pd.DataFrame(page["Rows"])['Data']]
            timeid = timeid + timeid_page
            data_values = data_values + data_values_page
        except KeyError:
            print('Page {%d} has no data available:'%i)
        i = i+1
    return timeid, data_values

In [ ]:

def build_dataframe(timeid, measure_name, data_values):
    # ============== Check if there is data available for the pv system =============
    if len(timeid)!=0:
        timeid = pd.to_datetime(timeid)
        if timeid.tzinfo is None:
            print('this is not tz-aware')
            if timezone_value is not None:
                timeid = timeid.tz_localize('UTC').tz_convert(timezone_value)
                # timeid = timeid.tz_localize(timezone_list[i])
            else:
                print('no timezone in the table')
                timeid = timeid.tz_localize('UTC').tz_convert('Australia/Sydney')
                # timeid = timeid.tz_localize('Australia/Sydney')
        else:
            print('this is tz-aware')
        
        timesort = timeid.sort_values()
        data = pd.DataFrame(data={'time':timeid, measure_name: data_values})
        data.sort_values('time', inplace=True)
        data.set_index('time', inplace=True)
        data[measure_name] = data[measure_name].astype(float)
    else:
        data = pd.DataFrame(data_values, index=timeid, columns=[measure_name])
    
    return data


In [ ]:
def change_tz(timeid):
    # print('rawtimeid:', timeid)
    tzinfo_str = timeid[0].tzinfo
    hour_offset = tzinfo_str.utcoffset(datetime.datetime(2022,1,1))
    hms = str(hour_offset).split(':')
    time_modified = timeid + datetime.timedelta(hours=int(hms[0]), minutes=int(hms[1]), seconds=int(hms[2]))
    time_utc = time_modified.dt.tz_convert('UTC')
    # print('modified:', time_utc)
    return time_utc

## 3.3. Setting up a 5minutes datetime dataframe

In [ ]:
time_index5min = pd.date_range(start=time_starttz, end=time_endtz, freq='5min').tz_convert('UTC')
df_5min = pd.DataFrame(index=np.arange(len(time_index5min)))
df_5min['time'] = time_index5min

# 4. AC Phase

## 4.1. Gen.W

In [ ]:
# ========================================================
# = Reading P(AC) total from AWS TimeStream
# = Metric is Gen.W 
# ========================================================

measure_name = 'Gen.W'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_genW = build_dataframe(timeid, measure_name, data_values)

In [ ]:
df_genW.index = df_genW.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_genW = pd.merge_asof(df_5min, df_genW, on="time", tolerance=pd.Timedelta("1 minute"))
df_genW

In [ ]:
fig = px.line(df_genW, x='time',  y=measure_name, title='P(AC)')
fig.show()

## 4.2. Vac per phase

In [ ]:
# ========================================================
# = Reading V(AC) average from AWS TimeStream
# = Metric is Inv.AC.U.V
# ========================================================

measure_name = 'Inv.AC.U.V'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_ac_voltage_avg = build_dataframe(timeid, measure_name, data_values)

In [ ]:
# ========================================================
# = Reading V(AC) per phase from AWS TimeStream
# = Metric is Inv.AC.U.V
# = Where X is a number between (1 - n)
# ========================================================
'''
phase=3
measure_name = 'Inv.AC.U.{' + str(phase) + '}V'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_ac_voltage_avg = build_dataframe(timeid, measure_name, data_values)
'''

In [ ]:
df_ac_voltage_avg.index = df_ac_voltage_avg.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_ac_voltage_avg = pd.merge_asof(df_5min, df_ac_voltage_avg, on="time", tolerance=pd.Timedelta("1 minute"))
df_ac_voltage_avg

In [ ]:
fig = px.line(df_ac_voltage_avg, y=measure_name, title=measure_name)
fig.show()

## 4.3. I(AC) per phase

In [ ]:
# ========================================================
# = Reading V(AC) per phase from AWS TimeStream
# = Metric is Inv.AC.I.Ph1.V 
# = Where X is a number between (1 - n)
# ========================================================

X=1
measure_name = 'Inv.AC.I.Ph{' + str(X) + '}.V'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_ac_current = build_dataframe(timeid, measure_name, data_values)

In [ ]:
df_ac_current.index = df_ac_current.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_ac_current = pd.merge_asof(df_5min, df_ac_current, on="time", tolerance=pd.Timedelta("1 minute"))
df_ac_current

In [ ]:
fig = px.line(df_ac_current, y=measure_name, title=measure_name)
fig.show()

## 4.4. P(AC) per phase

In [ ]:
# ========================================================
# = Reading V(AC) per phase from AWS TimeStream
# = Metric is Inv.AC.P.Ph1.V 
# = Where X is a number between (1 - n)
# ========================================================

X=1
measure_name = 'Inv.AC.P.Ph{' + str(X) + '}.V'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_ac_power = build_dataframe(timeid, measure_name, data_values)

In [ ]:
measure_name = 'Inv.AC.P.Ph.V'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_ac_power = build_dataframe(timeid, measure_name, data_values)

In [ ]:
df_ac_power.index = df_ac_power.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_ac_power = pd.merge_asof(df_5min, df_ac_power, on="time", tolerance=pd.Timedelta("1 minute"))
df_ac_power

In [ ]:
fig = px.line(df_ac_power, y=measure_name, title=measure_name)
fig.show()

# 5. DC Phase

## 5.1. Voltage (Inv.DC.U.V)

In [ ]:
# ========================================================
# = Reading V(DC) average from AWS TimeStream
# = Metric is Inv.DC.U.V
# ========================================================

measure_name = 'Inv.DC.U.V'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_dc_voltage = build_dataframe(timeid, measure_name, data_values)

In [ ]:
df_dc_voltage.index = df_dc_voltage.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_dc_voltage = pd.merge_asof(df_5min, df_dc_voltage, on="time", tolerance=pd.Timedelta("1 minute"))
df_dc_voltage

In [ ]:
fig = px.line(df_dc_voltage, y=measure_name, title='V(DC) - Avg')
fig.show()

## 5.3. Current (Inv.DC.I.A)

In [ ]:
# ========================================================
# = Reading I(DC) total from AWS TimeStream
# = Metric is Inv.DC.I.A 
# ========================================================

measure_name = 'Inv.DC.I.A'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_dc_current = build_dataframe(timeid, measure_name, data_values)

In [ ]:
df_dc_current.index = df_dc_current.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_dc_current = pd.merge_asof(df_5min, df_dc_current, on="time", tolerance=pd.Timedelta("1 minute"))
df_dc_current

In [ ]:
fig = px.line(df_dc_current, y="Inv.DC.I.A", title='I(DC) - Total')
fig.show()

## 5.4. Power (Inv.DC.P.W)

In [ ]:
# ========================================================
# = Reading P(DC) total from AWS TimeStream
# = Metric is Inv.DC.P.W
# ========================================================

measure_name = 'Inv.DC.P.W'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_dc_power = build_dataframe(timeid, measure_name, data_values)

In [ ]:
df_dc_power.index = df_dc_power.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_dc_power = pd.merge_asof(df_5min, df_dc_power, on="time", tolerance=pd.Timedelta("1 minute"))
df_dc_power

In [ ]:
fig = px.line(df_dc_power, y=measure_name, title='P(DC) total')
fig.show()

## 5.5. Resistance 

In [ ]:
# ========================================================
# = Reading R(DC) from AWS TimeStream
# = Metric is Inv.DC.R.Ohm
# ========================================================

measure_name = 'Inv.DC.R.Ohm'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_dc_resistance = build_dataframe(timeid, measure_name, data_values)

In [ ]:
df_dc_resistance.index = df_dc_resistance.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_dc_resistance = pd.merge_asof(df_5min, df_dc_resistance, on="time", tolerance=pd.Timedelta("1 minute"))
df_dc_resistance

In [ ]:
fig = px.line(df_dc_resistance, y=measure_name, title='P(DC) total')
fig.show()

# 6. PV Phase

## 6.1. Voltage

In [ ]:
# ========================================================
# = Reading Vpv from AWS TimeStream
# = Metric is Inv.DC.U.MPPT{X}.V
# = Where X is a number between (1 - n)
# ========================================================
X=1
measure_name = 'Inv.DC.U.MPPT{' + str(X) + '}.V'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_vPV = build_dataframe(timeid, measure_name, data_values)

In [ ]:
df_vPV.index = df_vPV.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_vPV = pd.merge_asof(df_5min, df_vPV, on="time", tolerance=pd.Timedelta("1 minute"))
df_vPV

In [ ]:
fig = px.line(df_vPV, y=measure_name, title='Vpv')
fig.show()

## 6.2. Current


In [ ]:
# ========================================================
# = Reading Ipv from AWS TimeStream
# = Metric is Inv.DC.I.MPPT{X}.A
# = Where X is a number between (1 - n)
# ========================================================

measure_name = 'Inv.DC.I.MPPT{' + str(X) + '}.A'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_iPV = build_dataframe(timeid, measure_name, data_values)


In [ ]:
df_iPV.index = df_iPV.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_iPV = pd.merge_asof(df_5min, df_iPV, on="time", tolerance=pd.Timedelta("1 minute"))
df_iPV

In [ ]:
fig = px.line(df_iPV, y=measure_name, title='Ipv')
fig.show()

## 6.3. Power

In [ ]:
# ========================================================
# = Reading Ppv from AWS TimeStream
# = Metric is Inv.DC.P.MPPT{X}.W
# = Where X is a number between (1 - n)
# ========================================================
X = 1
measure_name = 'Inv.DC.P.MPPT{' + str(X) + '}.W'
timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
df_pPV = build_dataframe(timeid, measure_name, data_values)

In [ ]:
df_pPV.index = df_pPV.index.tz_convert('UTC')
df_5min['time'] = df_5min['time'].dt.tz_convert('UTC')

In [ ]:
df_pPV = pd.merge_asof(df_5min, df_pPV, on="time", tolerance=pd.Timedelta("1 minute"))
df_pPV

In [ ]:
fig = px.line(df_pPV, y=measure_name, title='Ppv')
fig.show()

# Plotting

## Merging AC power and Voltage with DC power and voltage

In [ ]:
'''
This is a 3-phase system, hence 400V instead of 230V
'''
# Defining voltage limit:
# Voltage + 10%
v_limit_ac = (400*1.1)
v_limit_ac = 230*math.sqrt(3)*1.1

In [ ]:
df_ac_power_voltage = pd.merge(df_genW, df_ac_voltage_avg, on='time', how='outer')
df_ac_power_voltage['v_limit_ac'] = v_limit_ac
df_ac_power_voltage
#Might as well merge the datasets:
df_dc_power_voltage = pd.merge(df_dc_power, df_dc_voltage, on='time', how='outer')
df_dc_power_voltage
# Getting it all
df_all = pd.merge(df_ac_power_voltage, df_dc_power_voltage, on='time', how='outer')

In [ ]:
# Initialise the subplot function using (rows, columns)
figure, axis = plt.subplots(4, 1)
figure.set_size_inches(24, 10, forward=True)

# Setting up X-Axis
x = df_all.index

# Fig 1 - DC Voltage
axis[0].plot(x, df_all['Inv.DC.U.V'])
axis[0].set_title("DC Voltage")


# Fig 2 - DC Power
axis[1].plot(x, df_all['Inv.DC.P.W'])
axis[1].set_title("DC Power")

# Fig 3 - AC Voltage + limit
axis[2].plot(x, df_all['Inv.AC.U.V'])
axis[2].plot(x, df_all['v_limit_ac'])
axis[2].set_title("AC Voltage")

# Fig 4 - AC Power
axis[3].plot(x, df_all['Gen.W'])
axis[3].set_title("AC Power")

# Combine all the operations and display
plt.savefig('./plots/curtailment/Curtailment_Check_MID' + str(MID) + '_' + str(time_start) + '_to_' + str(time_end))
plt.show()